In [ ]:
# importing the module 
import sys
import os
import shutil
import pandas as pd
from moviepy.editor import *
import pytube 
from google.colab import drive
drive.mount('/content/drive')


def read_csv(PATH,label):
    try:
        df = pd.read_csv(PATH)
    except:
        print('ERROR :File not found, check file path again')
      
    lables = df['label'].tolist()
    youtube_ids = df['youtube_id'].tolist()
    time_start = df['time_start'].tolist()
    time_end = df['time_end'].tolist()
    l = []

    for i in range(len(lables)):
        if label in lables[i]:
            l.append(lables[i] + "," + f'https://www.youtube.com/watch?v={youtube_ids[i]}' + "," + str(time_start[i]) + "," + str(time_end[i]))    
    return l

def clip(path,file_name,start_time,end_time):
    video = VideoFileClip(f'{path}/{file_name}.mp4').subclip(start_time,end_time)
    frames = video.fps
    video.write_videofile(f'{path}/clipped/{file_name}.mp4',fps=frames)

def remove_special_characters(name):
    l = ["\\","/","*",":","|","?","<",">",'"']
    for letter in l:
        if letter in name:
            name = name.replace(letter,"")
    return name

def downloadVideo(urls,label):
    cwd = 'drive/My Drive/Colab Notebooks' 
    PATH = f'{cwd}/{label} videos'
    print(f'Total Links found :{len(urls)}')
    try:
      file = open(f'{cwd}/{label}.txt' , 'r')
      x = int(file.read())
      file.close()
    except FileNotFoundError:
      x = 0
    print("Starting from",x)
    for i in range(x,len(urls)):
        file = open(f'{cwd}/{label}.txt' , 'w')
        file.write(str(i))
        file.close()
        url = urls[i]   
        try:
            youtube = pytube.YouTube(url.split(',')[1])
            video = youtube.streams.get_highest_resolution()
            print(f'Downloading {video.title} | size = {video.filesize/1000000} mb')
            video.download(filename = f'{PATH}/{remove_special_characters(video.title)}.mp4')
            if youtube.length > 10:
                clip(PATH,remove_special_characters(video.title),int(url.split(',')[2]),int(url.split(',')[3]))
            else:
                original = f'{PATH}/{remove_special_characters(video.title)}.mp4'
                target = f'{PATH}/clipped\\{remove_special_characters(video.title)}.mp4'
                shutil.copyfile(original, target)
        except Exception as e: print(e)
    
if __name__ == "__main__":
    LABEL = 'ADD YOUR LABEL HERE' # <===================================== ADD YOUR LABEL HERE
    cwd = 'drive/My Drive/Colab Notebooks' # change your current directory of google drive if its different
    try:
        os.mkdir(f'{cwd}/{LABEL} videos')
    except:
        pass
    try:
        os.mkdir(f'{cwd}/{LABEL} videos/clipped')
    except:
        pass
    urls = read_csv(f'{cwd}/train.csv',LABEL)
    downloadVideo(urls,LABEL)